# Завантаження бібліотек

In [3]:
import numpy as np
import pandas as pd
import cv2
import time
import pickle

from keras.models import load_model
from matplotlib import pyplot as plt

# Підготовка і завантаження моделі

In [55]:
data_path = './DetectionData'
preprocessed_path = data_path + "/traffic-signs-preprocessed"
in_yolo_format_path = data_path + "/traffic-signs-dataset-in-yolo-format"
signs_detector_path = data_path + "/trained-traffic-signs-detector-based-on-yolo-v3"

cls_model = load_model("./sign_classifier_v1m4.h5")
input_shape = cls_model.layers[0].input_shape[1:3]
print("Input shape: ", input_shape)

class_labels = pd.read_csv("./ClassificationDataSet/Labels.csv")

with open(preprocessed_path + '/mean_image_rgb.pickle', 'rb') as f:
    mean = pickle.load(f, encoding='latin1')

print("Classificator model loaded")
print("Mean image shape: ", mean['mean_image_rgb'].shape)

Input shape:  (32, 32)
Classificator model loaded
Mean image shape:  (3, 32, 32)


In [56]:
path_to_weights = signs_detector_path + "/yolov3_ts_train_5000.weights"
path_to_cfg = in_yolo_format_path + "/yolov3_ts_test.cfg"

det_model = cv2.dnn.readNetFromDarknet(path_to_cfg, path_to_weights)
det_model.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
det_model.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL_FP16)

print("Detection model loaded")

Detection model loaded


In [57]:
layers_all = det_model.getLayerNames()
layers_names_output = [layers_all[i - 1] for i in det_model.getUnconnectedOutLayers()]
print(layers_names_output)

['yolo_82', 'yolo_94', 'yolo_106']


In [58]:
probability_minimum = 0.1
threshold = 0.2

# Завантаження зображення

In [144]:
image = cv2.imread("./MyTestData/Detection/full_photo2.jpg")

h, w = image.shape[:2]
print("Image shape: ", image.shape)

Image shape:  (637, 1280, 3)


In [145]:
blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
det_model.setInput(blob)
start = time.time()
output_from_detector = det_model.forward(layers_names_output)
end = time.time()

print('Objects Detection took {:.5f} seconds'.format(end - start))

Objects Detection took 0.85267 seconds


In [146]:
bounding_boxes = []
confidences = []
class_numbers = []

# Going through all output layers after feed forward pass
for result in output_from_detector:
    # Going through all detections from current output layer
    for detection in result:
        score = detection[5:]
        class_current = np.argmax(score)
        confidence_current = score[class_current]

        # Eliminating weak predictions with minimum probability
        if confidence_current > probability_minimum:
            box_current = detection[0:4] * np.array([w, h, w, h])

            # Now, from YOLO data format, convert it to format that is used by OpenCV
            x_center, y_center, box_width, box_height = box_current.astype('int')
            box_width *= 1.1
            box_height *= 1.1
            x_min = int(x_center - (box_width / 2))
            y_min = int(y_center - (box_height / 2))

            # Adding results into prepared lists
            bounding_boxes.append([x_min, y_min, int(box_width), int(box_height)])
            confidences.append(float(confidence_current))
            class_numbers.append(class_current)


In [147]:
results = cv2.dnn.NMSBoxes(bounding_boxes, confidences, probability_minimum, threshold)

if len(results) > 0:
    for i in results.flatten():
        x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
        box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]

        # cut detected sign
        cut_sign = image[y_min:y_min + box_height, x_min:x_min + box_width]

        blob_sign = cv2.dnn.blobFromImage(cut_sign, 1 / 255.0, input_shape, swapRB=False, crop=False)
        blob_sign[0] -= mean['mean_image_rgb']
        blob_sign = blob_sign.transpose(0, 2, 3, 1)

        # classify sign
        class_predicted = cls_model.predict(blob_sign)[0]
        inID = np.argmax(class_predicted)
        label = class_labels.loc[inID][1] + ': ' + str(round(class_predicted[inID], 2))

        colour_box_current = (255, 255, 255)
        cv2.rectangle(image, (x_min, y_min), (x_min + box_width, y_min + box_height), colour_box_current, 2)

        text_box_current = label
        # add shadow to text
        (text_width, text_height) = cv2.getTextSize(text_box_current, cv2.FONT_HERSHEY_TRIPLEX, 0.7, 1)[0]
        text_offset_x = x_min
        text_offset_y = y_min - 5
        box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 2, text_offset_y - text_height))
        overlay = image.copy()
        cv2.rectangle(overlay, box_coords[0], box_coords[1], (0, 0, 0), cv2.FILLED)
        image = cv2.addWeighted(overlay, 0.6, image, 0.4, 0)
        # add text
        cv2.putText(image, text_box_current, (x_min, y_min - 5), cv2.FONT_HERSHEY_TRIPLEX, 0.7, colour_box_current, 1, cv2.LINE_AA)

        print("Detected: ", label)

    cv2.imwrite("./classification_and_detection.jpg", image)

else:
    print('No signs detected')

1/1 [==============================] - 0s 17ms/step
Detected:  No entry: 1.0


In [148]:
cv2.imshow('Detection results', image)
cv2.waitKey(0)

-1